<h1 style="text-align:center;">🎮 Gamer Performance Analyzer</h1>
<h3 style="text-align:center;">by Zuhayr Chowdhury</h3>
<p style="text-align:center;">
An end-to-end analysis of gaming habits to determine whether performance improves with more playtime, or if it's just grind.
</p>
<hr>


<h1>📁 SECTION 1: Load Gamer Data into SQLite</h1>
<p>
This section reads the <code>gamer_habits_data.csv</code> file, renames necessary columns, 
creates a <code>gamer_sessions</code> table in a local SQLite database, 
and inserts all the data for analysis.
</p>


In [36]:
import pandas as pd
import sqlite3

# Load your generated data from CSV
df = pd.read_csv("gamer_habits_data.csv")

# Connect to SQLite and create table
conn = sqlite3.connect("gamer_habits.db")
cursor = conn.cursor()

# Drop old table if re-running
cursor.execute("DROP TABLE IF EXISTS gamer_sessions")

# Create new table
cursor.execute("""
CREATE TABLE gamer_sessions (
    Date TEXT,
    Game TEXT,
    Session_Hours REAL,
    Matches_Played INTEGER,
    Wins INTEGER,
    Kills INTEGER,
    Deaths INTEGER,
    KD_Ratio REAL,
    Mood TEXT
)
""")

# Insert DataFrame into SQL table
df.to_sql("gamer_sessions", conn, if_exists="append", index=False)

# Test query
result = pd.read_sql_query("SELECT * FROM gamer_sessions LIMIT 5", conn)
conn.close()

# Show result
result


,Date,Game,Session_Hours,Matches_Played,Wins,Kills,Deaths,KD_Ratio,Mood
0,2025-02-08,Apex Legends,0.7,1,1,5,23,0.22,Tired
1,2025-02-09,Overwatch,2.0,4,1,30,14,2.14,Focused
2,2025-02-10,Valorant,2.9,4,1,12,4,3.00,Tired
3,2025-02-11,Valorant,2.4,2,0,22,9,2.44,Frustrated
4,2025-02-12,Overwatch,2.2,3,0,6,16,0.38,Focused


<h1>📊 SECTION 2: Analyze Gamer Habits & Performance</h1>
<p>
This section runs a series of SQL queries to explore gaming patterns, 
such as average K/D ratio, win rate trends, mood impact, and session fatigue. 
Each query returns insight into the player's grind versus skill performance.
</p>


In [37]:
# 1. Average K/D Ratio per Game

query = """
SELECT Game, ROUND(AVG(KD_Ratio), 2) AS Avg_KD
FROM gamer_sessions
GROUP BY Game
ORDER BY Avg_KD DESC
"""
pd.read_sql_query(query, sqlite3.connect("gamer_habits.db"))


,Game,Avg_KD
0,Apex Legends,2.33
1,Overwatch,1.52
2,Valorant,1.45


In [38]:
# 2. Total Wins per Game
query = """
SELECT Game, SUM(Wins) AS Total_Wins
FROM gamer_sessions
GROUP BY Game
ORDER BY Total_Wins DESC
"""
pd.read_sql_query(query, sqlite3.connect("gamer_habits.db"))

,Game,Total_Wins
0,Overwatch,52
1,Valorant,30
2,Apex Legends,29


In [39]:
# 3. Most Productive Mood (based on K/D)
query = """
SELECT Mood, ROUND(AVG(KD_Ratio), 2) AS Avg_KD
FROM gamer_sessions
GROUP BY Mood
ORDER BY Avg_KD DESC
"""
pd.read_sql_query(query, sqlite3.connect("gamer_habits.db"))

,Mood,Avg_KD
0,Frustrated,2.16
1,Happy,1.78
2,Casual,1.59
3,Tired,1.54
4,Focused,1.48


In [40]:
# 4. Session Hours vs. Performance (Win Rate)
query = """
SELECT Session_Hours,
       ROUND(AVG(CAST(Wins AS FLOAT) / Matches_Played), 2) AS Win_Rate
FROM gamer_sessions
GROUP BY Session_Hours
ORDER BY Session_Hours
"""
pd.read_sql_query(query, sqlite3.connect("gamer_habits.db"))

,Session_Hours,Win_Rate
0,0.7,0.75
1,0.8,1.00
2,1.0,0.00
3,1.6,0.50
4,1.7,0.17
5,2.0,0.42
6,2.2,0.00
7,2.3,0.67
8,2.4,0.17
9,2.5,0.55


In [41]:
# 5. Weekly Game Trends
query = """
SELECT strftime('%w', Date) AS Weekday,
       Game,
       COUNT(*) AS Sessions
FROM gamer_sessions
GROUP BY Weekday, Game
ORDER BY Weekday, Sessions DESC
"""
pd.read_sql_query(query, sqlite3.connect("gamer_habits.db"))


,Weekday,Game,Sessions
0,0,Valorant,5
1,0,Overwatch,3
2,0,Apex Legends,1
3,1,Valorant,5
4,1,Overwatch,3
5,1,Apex Legends,1
6,2,Apex Legends,4
7,2,Overwatch,3
8,2,Valorant,2
9,3,Overwatch,5


In [42]:
# 6. Win Rate Per Game
pd.read_sql_query("""
SELECT 
  Game, 
  ROUND(SUM(Wins) * 1.0 / SUM(Matches_Played), 2) AS Win_Rate
FROM gamer_sessions
GROUP BY Game
ORDER BY Win_Rate DESC
""", sqlite3.connect("gamer_habits.db"))


,Game,Win_Rate
0,Apex Legends,0.63
1,Overwatch,0.51
2,Valorant,0.33


In [43]:
# 7. Average Session Length Per Game
pd.read_sql_query("""
SELECT 
  Game, 
  ROUND(AVG(Session_Hours), 2) AS Avg_Session_Length
FROM gamer_sessions
GROUP BY Game
ORDER BY Avg_Session_Length DESC
""", sqlite3.connect("gamer_habits.db"))

,Game,Avg_Session_Length
0,Overwatch,3.47
1,Valorant,3.22
2,Apex Legends,2.61


In [44]:
# 8. Performance Trend (K/D Over Time)
pd.read_sql_query("""
SELECT 
  Date, 
  ROUND(AVG(KD_Ratio), 2) AS Avg_KD
FROM gamer_sessions
GROUP BY Date
ORDER BY Date
""", sqlite3.connect("gamer_habits.db"))


,Date,Avg_KD
0,2025-02-08,0.22
1,2025-02-09,2.14
2,2025-02-10,3.00
3,2025-02-11,2.44
4,2025-02-12,0.38
5,2025-02-13,0.38
6,2025-02-14,1.50
7,2025-02-15,1.45
8,2025-02-16,1.60
9,2025-02-17,2.80


In [45]:
# 9. Session Length vs. Win Rate
pd.read_sql_query("""
SELECT 
  ROUND(Session_Hours, 1) AS Session_Length, 
  ROUND(AVG(CAST(Wins AS FLOAT) / Matches_Played), 2) AS Win_Rate
FROM gamer_sessions
GROUP BY Session_Length
ORDER BY Session_Length
""", sqlite3.connect("gamer_habits.db"))


,Session_Length,Win_Rate
0,0.7,0.75
1,0.8,1.00
2,1.0,0.00
3,1.6,0.50
4,1.7,0.17
5,2.0,0.42
6,2.2,0.00
7,2.3,0.67
8,2.4,0.17
9,2.5,0.55


In [46]:
# 10. Mood vs. Win Rate & K/D
pd.read_sql_query("""
SELECT 
  Mood, 
  ROUND(AVG(CAST(Wins AS FLOAT) / Matches_Played), 2) AS Win_Rate,
  ROUND(AVG(KD_Ratio), 2) AS Avg_KD
FROM gamer_sessions
GROUP BY Mood
ORDER BY Win_Rate DESC
""", sqlite3.connect("gamer_habits.db"))


,Mood,Win_Rate,Avg_KD
0,Focused,0.58,1.48
1,Happy,0.54,1.78
2,Casual,0.53,1.59
3,Tired,0.49,1.54
4,Frustrated,0.49,2.16


In [47]:
# 11. Most Played Game
pd.read_sql_query("""
SELECT 
  Game, 
  COUNT(*) AS Session_Count
FROM gamer_sessions
GROUP BY Game
ORDER BY Session_Count DESC
""", sqlite3.connect("gamer_habits.db"))


,Game,Session_Count
0,Overwatch,23
1,Valorant,21
2,Apex Legends,16


In [48]:
# 12. Most Active Day of the Week

pd.read_sql_query("""
SELECT 
  CASE strftime('%w', Date)
    WHEN '0' THEN 'Sunday'
    WHEN '1' THEN 'Monday'
    WHEN '2' THEN 'Tuesday'
    WHEN '3' THEN 'Wednesday'
    WHEN '4' THEN 'Thursday'
    WHEN '5' THEN 'Friday'
    WHEN '6' THEN 'Saturday'
  END AS Weekday,
  COUNT(*) AS Sessions
FROM gamer_sessions
GROUP BY Weekday
ORDER BY Sessions DESC
""", sqlite3.connect("gamer_habits.db"))


,Weekday,Sessions
0,Tuesday,9
1,Sunday,9
2,Saturday,9
3,Monday,9
4,Wednesday,8
5,Thursday,8
6,Friday,8
